<a href="https://colab.research.google.com/github/TammyJade/TJ/blob/main/EFT_Tech_Companies_Stock_Watch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen       # defines functions and classes which help in opening URLs
import re                                # supports various things like Modifiers, Identifiers, and White space characters.
import requests
import pandas as pd

In [ ]:
# User agents: retrieve, render and facilitate end-user interaction with Web content. A user agent is a special kind of software agent

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"  # user agent. 

LANGUAGE = "en-US,en;q=0.5"  # US english

# Constructs and returns a soup using the HTML content of `url` passed

def get_soup(url):
    session = requests.Session()                    # Initializing session
    session.headers['User-Agent'] = USER_AGENT      # Setting User-Agent as regular browser
    session.headers['Accept-Language'] = LANGUAGE   # Specifying english 
    session.headers['Content-Language'] = LANGUAGE  # Specifying english
    html = session.get(url)                         # Getting the url- similar to - (requests.get(url))
    return bs(html.content, "html.parser")          # Returning the soup content
    
def get_all_tables(soup):                           # Extracts and returns all tables in a soup object
    return soup.find_all("table")

def get_table_headers(table):                       # Returns the headers given a table soup 
    headers = []
    for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())
    return headers

def get_table_rows(table):                           # Given a table, returns all its rows
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = []
        tds = tr.find_all("td")                      # Finding all td tags in this table row
        if len(tds) == 0:                            # if no td tags search for th tags
            ths = tr.find_all("th")                
            for th in ths:
                cells.append(th.text.strip())
        else:
            for td in tds:                           # Else use regular td tags
                cells.append(td.text.strip())
        rows.append(cells)
    return rows

def save_as_csv(table_name, headers, rows):
    pd.DataFrame(rows, columns=headers).to_csv(f"{table_name}.csv")

def main(url):   
    soup = get_soup(url)                         # Get the soup - parse html content through beautifulsoup
    tables = get_all_tables(soup)                # Extract ALL tables from web page get_all_tables
    print(f"Total of {len(tables)} tables.")
    for i, table in enumerate(tables, start=1):  # For loop iterating over all tables- enumerate() assigns index to each item in an iterable object that can be used to reference the item later. 
        headers = get_table_headers(table)       # Get the table headers
        rows = get_table_rows(table)             # Get all the rows of the table
        table_name = f"table-{i}"                # Saving table as csv 
        print(f"Saving {table_name}")
        save_as_csv(table_name, headers, rows)

main("https://etfdb.com/etfdb-category/technology-equities/#etfs__overview&sort_name=symbol&sort_order=asc&page=1")   # The web-page with the tables being scraped

Total of 1 tables.
Saving table-1


In [ ]:
df1 = pd.read_csv('table-1.csv')  # Reading the saved csv
df1.tail()                        # showing top 5 rows   

,Unnamed: 0,Symbol,ETF Name,Total Assets*,YTD,Avg Volume,Previous Closing Price,1-Day Change,1 Week,4 Week,1 year,3 year,5 year,YTD FF,1 Week FF,4 Week FF,1 Year FF,3 Year FF,5 Year FF,ETFdb.com Category,Inception,ER,Commission Free,Annual Dividend Rate,Dividend Date,Dividend,Annual Dividend Yield %,P/E Ratio,Beta,# of Holdings,% In Top 10,Complete,ST Cap Gain Rate,LT Cap Gain Rate,Tax Form,Lower Bollinger,Upper Bollinger,Support 1,Resistance 1,RSI,Advanced,Fact Sheet,ETF Holdings,Chart,ETF Home Page,Head-To-Head,Overall Rating,Liquidity Rating,Expenses Rating,Returns Rating,Volatility Rating,Dividend Rating,Concentration Rating,ESG Score,ESG Score Peer Percentile (%),ESG Score Global Percentile (%),Carbon Intensity (Tons of CO2e / $M Sales),SRI Exclusion Criteria (%),Sustainable Impact Solutions (%)
21,21,BLOK,Amplify Transformational Data Sharing ETF,"$1,031,510.00",33.68%,"989,958.0",$46.76,1.30%,5.86%,-9.01%,150.16%,143.54%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Technology Equities,2018-01-16,0.70%,NaN,$0.66,2020-12-29,$0.66,1.43%,30.06,1.29,47.0,40.63%,View,39%,20%,1099.0,$40.19,$53.49,$46.23,$47.13,46.58,View,View,View,View,View,View,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,XSD,SPDR S&P Semiconductor ETF,"$892,444.00",4.18%,"129,450.0",$177.85,1.01%,3.79%,-3.53%,65.87%,141.11%,311.38%,NaN,NaN,NaN,NaN,NaN,NaN,Technology Equities,2006-01-31,0.35%,NaN,$0.40,2021-03-22,$0.08,0.23%,31.28,1.32,42.0,29.12%,View,35%,15%,1099.0,$158.52,$180.70,$176.09,$179.12,52.31,View,View,View,View,View,View,NaN,A-,A-,NaN,NaN,NaN,NaN,5.27,52.22%,34.14%,100.59,0.00%,9.71%
23,23,XNTK,SPDR NYSE Technology ETF,"$662,704.00",5.01%,"26,009.0",$148.24,0.34%,2.13%,-2.41%,64.63%,109.99%,270.77%,NaN,NaN,NaN,NaN,NaN,NaN,Technology Equities,2000-09-25,0.35%,NaN,$0.48,2021-03-22,$0.18,0.33%,31.22,1.18,36.0,33.89%,View,39%,20%,1099.0,$136.32,$151.83,$147.56,$148.66,53.56,View,View,View,View,View,View,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.99,70.99%,50.79%,36.12,0.00%,8.78%
24,24,XSW,SPDR S&P Software & Services ETF,"$607,514.00",4.44%,"39,556.0",$160.94,0.43%,2.87%,-2.97%,53.98%,102.40%,226.67%,NaN,NaN,NaN,NaN,NaN,NaN,Technology Equities,2011-09-28,0.35%,NaN,$0.08,2021-03-22,$0.00,0.05%,31.23,1.18,182.0,7.08%,View,35%,15%,1099.0,$149.13,$163.45,$159.48,$161.86,53.13,View,View,View,View,View,View,NaN,C,A,NaN,NaN,NaN,NaN,4.94,36.17%,29.54%,11.46,0.57%,2.32%
25,25,Click Here to Join to ETFdb Pro for 14 Days Fr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df1.drop([25], inplace = True)

In [ ]:
df2 = df1['Symbol']         # Creaing a df -will use the symbols to call the api 
df2 = pd.DataFrame(df2)     # Removing last row that wasnt applicable 

In [ ]:
list_of_symbols = df2['Symbol'].to_list()    # Chaning the df of symbols from the table to a list to itterate over.
print(list_of_symbols)

['VGT', 'XLK', 'IYW', 'SOXX', 'SKYY', 'FTEC', 'IXN', 'SMH', 'IGV', 'CIBR', 'QTEC', 'IGM', 'RYT', 'FXL', 'HACK', 'TDIV', 'CLOU', 'IPAY', 'FIVG', 'FINX', 'WCLD', 'BLOK', 'XSD', 'XNTK', 'XSW']


In [ ]:
pip install alpha_vantage  # Installing API

     |████████████████████████████████| 1.3MB 19.2MB/s 
     |████████████████████████████████| 143kB 48.0MB/s 
     |████████████████████████████████| 296kB 58.2MB/s 


In [ ]:
import requests
import json
import time                                                    # Can only make 5 calls to api per min

from alpha_vantage.timeseries import TimeSeries                # api documentation for time series data
from pprint import pprint
ts = TimeSeries(key='', output_format='json')                  # Insert unique api key, specifying output format 

In [ ]:
# Extracting the key information from json and inserting them into separate lists

symbol = []
open = []
high = []
low = []
price = []
volume = []
latest_trading_day = []
previous_close = []
change = []
change_percent = []

for x in list_of_symbols:                           # Itterating over the list and calling API for each element in the companies list 
  symbol_endpoint = ts.get_quote_endpoint(x) 

  symbol.append(symbol_endpoint[0]["01. symbol"])   # Appending the key financial information to the empty lists. 
  open.append(symbol_endpoint[0]["02. open"])
  high.append(symbol_endpoint[0]["03. high"])
  low.append(symbol_endpoint[0]["04. low"])
  price.append(symbol_endpoint[0]["05. price"])
  volume.append(symbol_endpoint[0]["06. volume"])
  latest_trading_day.append(symbol_endpoint[0]["07. latest trading day"])
  previous_close.append(symbol_endpoint[0]["08. previous close"])
  change.append(symbol_endpoint[0]["09. change"])
  change_percent.append(symbol_endpoint[0]["10. change percent"])

  json.dumps(symbol_endpoint, sort_keys=True, indent= 10)   # we can use it to print a formatted string which makes it easier to understand the JSON output
  time.sleep(65)                                            # sort_keys -re-arranges the keys in the json object in alphabetical/numeric order

In [ ]:
#creating a dictionary which will later be converted to a dataframe

endpoint = { 
          "01. symbol": symbol,
          "02. open": open,
          "03. high": high,
          "04. low": low,
          "05. price": price,
          "06. volume": volume,
          "07. latest trading day": latest_trading_day,
          "08. previous close": previous_close,
          "09. change": change,
          "10. change percent":change_percent
        }

In [ ]:
endpoint_df = pd.DataFrame(endpoint)  # Creating the dataframe from the dictionary                    

In [ ]:
endpoint_df["02. open"] = pd.to_numeric(endpoint_df["02. open"])   # Changing these columns from object type to numeric to get summary stats
endpoint_df["03. high"] = pd.to_numeric(endpoint_df["03. high"])
endpoint_df["04. low"] = pd.to_numeric(endpoint_df["04. low"])
endpoint_df["05. price"] = pd.to_numeric(endpoint_df["05. price"])
endpoint_df["06. volume"] = pd.to_numeric(endpoint_df["06. volume"])
endpoint_df["08. previous close"] = pd.to_numeric(endpoint_df["08. previous close"])
endpoint_df["09. change"] = pd.to_numeric(endpoint_df["09. change"])

In [ ]:
endpoint_df.drop(["01. symbol"], axis=1, inplace = True)

In [ ]:
endpoint_df.tail()

,02. open,03. high,04. low,05. price,06. volume,07. latest trading day,08. previous close,09. change,10. change percent
20,49.74,50.34,49.51,49.56,461754,2021-05-28,49.2100,0.3500,0.7112%
21,46.36,46.78,45.72,45.82,331154,2021-05-28,46.7600,-0.9400,-2.0103%
22,178.80,179.52,178.01,178.76,18631,2021-05-28,177.8500,0.9100,0.5117%
23,148.64,150.05,148.64,149.02,28797,2021-05-28,148.2361,0.7839,0.5288%
24,161.64,162.06,160.71,160.71,24919,2021-05-28,160.9400,-0.2300,-0.1429%


In [ ]:
# Dropping unwanted columns

df1.drop(['Unnamed: 0','Total Assets*','YTD','Avg Volume','Previous Closing Price','1-Day Change','1 Week','4 Week','1 year','3 year', '5 year', 'YTD FF', 'ER', 'Commission Free', 
          'Annual Dividend Rate', 'Dividend', 'Annual Dividend Yield %', '% In Top 10', 'Complete', 'ST Cap Gain Rate', 'LT Cap Gain Rate', 'Tax Form', 'Lower Bollinger', 
          'Upper Bollinger', 'Support 1', 'Resistance 1','Advanced', 'Fact Sheet', 'ETF Holdings', 'Chart', 'ETF Home Page', 'Head-To-Head',  'ESG Score Peer Percentile (%)', 
          'ESG Score Global Percentile (%)','Overall Rating', 'Returns Rating','Volatility Rating', 'Dividend Rating', 'Concentration Rating', '1 Week FF', '4 Week FF', 
          '1 Year FF','3 Year FF','5 Year FF','SRI Exclusion Criteria (%)','Sustainable Impact Solutions (%)'], axis=1, inplace = True)  

In [ ]:
df1.tail()

,Symbol,ETF Name,ETFdb.com Category,Inception,Dividend Date,P/E Ratio,Beta,# of Holdings,RSI,Liquidity Rating,Expenses Rating,ESG Score,Carbon Intensity (Tons of CO2e / $M Sales)
20,WCLD,WisdomTree Cloud Computing Fund,Technology Equities,2019-09-06,NaN,0.00,1.17,59.0,51.13,NaN,NaN,4.81,11.00
21,BLOK,Amplify Transformational Data Sharing ETF,Technology Equities,2018-01-16,2020-12-29,30.06,1.29,47.0,46.58,NaN,NaN,NaN,NaN
22,XSD,SPDR S&P Semiconductor ETF,Technology Equities,2006-01-31,2021-03-22,31.28,1.32,42.0,52.31,A-,A-,5.27,100.59
23,XNTK,SPDR NYSE Technology ETF,Technology Equities,2000-09-25,2021-03-22,31.22,1.18,36.0,53.56,NaN,NaN,5.99,36.12
24,XSW,SPDR S&P Software & Services ETF,Technology Equities,2011-09-28,2021-03-22,31.23,1.18,182.0,53.13,C,A,4.94,11.46


In [ ]:
df3 = pd.concat([df1, endpoint_df], axis=1).reindex(df1.index)  # Merging the table from the web and the table created from the API - with the selected columns

In [ ]:
df3.head() 

,Symbol,ETF Name,ETFdb.com Category,Inception,Dividend Date,P/E Ratio,Beta,# of Holdings,RSI,Liquidity Rating,Expenses Rating,ESG Score,Carbon Intensity (Tons of CO2e / $M Sales),02. open,03. high,04. low,05. price,06. volume,07. latest trading day,08. previous close,09. change,10. change percent
0,VGT,Vanguard Information Technology ETF,Technology Equities,2004-01-26,2021-03-26,35.40,1.10,332.0,51.82,A,A,5.76,22.73,372.47,373.8541,371.13,371.28,237555,2021-05-27,372.85,-1.57,-0.4211%
1,XLK,Technology Select Sector SPDR Fund,Technology Equities,1998-12-16,2021-03-22,29.23,1.06,76.0,51.74,A+,A,6.15,22.26,138.52,139.0900,137.88,137.91,5677344,2021-05-27,138.64,-0.73,-0.5265%
2,IYW,iShares U.S. Technology ETF,Technology Equities,2000-05-15,2021-03-25,36.83,1.08,161.0,53.81,A,A-,4.44,21.88,92.51,92.8446,92.20,92.22,159649,2021-05-27,92.61,-0.39,-0.4211%
3,SOXX,iShares PHLX Semiconductor ETF,Technology Equities,2001-07-10,2021-03-25,37.08,1.23,32.0,54.37,A,B+,8.75,73.74,426.11,430.8500,425.00,428.70,1100909,2021-05-27,426.90,1.80,0.4216%
4,SKYY,First Trust Cloud Computing ETF,Technology Equities,2011-07-05,2021-03-25,28.69,1.01,67.0,51.05,B,B,4.99,16.74,98.00,98.5800,97.11,98.10,167846,2021-05-27,98.31,-0.21,-0.2136%


In [ ]:
df3.describe() # Summary Stats

,P/E Ratio,Beta,# of Holdings,RSI,ESG Score,Carbon Intensity (Tons of CO2e / $M Sales),02. open,03. high,04. low,05. price,06. volume,08. previous close,09. change
count,25.00000,25.000000,25.000000,25.000000,24.000000,24.000000,25.000000,25.000000,25.000000,25.000000,2.500000e+01,25.000000,25.000000
mean,34.78000,1.128000,101.640000,53.154000,6.139167,32.965833,160.111600,161.037312,159.404420,160.109200,6.013273e+05,160.252244,-0.143044
std,15.12365,0.109163,91.715538,2.528297,1.384696,25.380882,125.363449,126.110221,124.903013,125.411212,1.209591e+06,125.616497,0.949907
min,0.00000,0.980000,26.000000,46.580000,4.440000,7.710000,26.110000,26.271500,25.700000,26.110000,1.260100e+04,26.270000,-2.500000
25%,29.65000,1.040000,42.000000,51.740000,4.982500,15.852500,56.330000,56.688400,56.330000,56.510000,3.919700e+04,56.330000,-0.640000
50%,31.28000,1.100000,67.000000,52.900000,5.855000,22.995000,118.340000,119.252000,117.950000,118.860000,1.868330e+05,118.370000,-0.090000
75%,37.08000,1.210000,111.000000,54.370000,6.840000,36.195000,244.780000,247.410000,244.200000,246.430000,4.617540e+05,245.250000,0.350000
max,90.05000,1.320000,332.000000,58.530000,9.910000,100.590000,426.110000,430.850000,425.000000,428.700000,5.677344e+06,426.900000,1.800000


In [ ]:
from google.colab import files     # exporting csv
df3.to_csv('Tech_Stock_Watch.csv') 
files.download('Tech_Stock_Watch.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>